<a href="https://colab.research.google.com/github/maimuna-noshin/Solar_Prediction/blob/main/solarprediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [219]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import get_custom_objects
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import pickle
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint
from tensorflow.keras.layers import Dense, Dropout, LSTM, Conv1D, BatchNormalization, Flatten, Bidirectional, LayerNormalization, Input
from sklearn.preprocessing import StandardScaler
from scipy.fftpack import fft
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.models import Sequential, Model

In [220]:
df=pd.read_csv("/content/updated_dataset.csv")

In [221]:
# Convert to datetime
df['date'] = pd.to_datetime(df[['Year', 'Month', 'Day', 'Hour']])




In [222]:
# Selecting features (X) and target variables (Y)
features = ['Hour', 'Temperature_A','Pressure_A', 'Day','Month', 'Surface Albedo_A', 'Cloud Type_A']
targets = ['DNI_A']  # Target variables for all three locations

In [223]:

X = df[features].values
y = df[targets].values

In [224]:
# Keep timestamps before scaling
timestamps = df[['Month', 'Day', 'Hour']]

In [225]:
scaler_X = MinMaxScaler()
# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
scaler_y = MinMaxScaler(feature_range=(0, 1000))
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)
# Save the feature scaler
with open("scaler_X.pkl", "wb") as f:
    pickle.dump(scaler_X, f)

# Save the target scaler
with open("scaler_y.pkl", "wb") as f:
    pickle.dump(scaler_y, f)
# Create binary classification labels (0 if DNI_A == 0, 1 if DNI_A > 0)
y_class = (y > 0).astype(int)


In [226]:
hours_tensor = tf.convert_to_tensor(df['Hour'].values, dtype=tf.float32)


In [227]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test, timestamps_train, timestamps_test = train_test_split(X_scaled, y_scaled,timestamps, test_size=0.2, random_state=42)

# Correct Boolean Masking for Selecting Nonzero DNI_A Cases
daytime_mask = y > 0  # Creates a boolean mask where DNI_A > 0
X_day = X_scaled[daytime_mask.flatten()]  # Apply mask to X_scaled
y_day = y[daytime_mask]  # Apply mask to y

# Split Data for Regression
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X_day, y_day, test_size=0.2, random_state=42)


In [228]:
def custom_loss_fixed(y_true, y_pred):
    batch_size = tf.shape(y_true)[0]
    batch_hours = tf.gather(hours_tensor, tf.range(batch_size))  # Use hours_tensor instead of hours

    nighttime_mask = tf.logical_or(tf.less_equal(batch_hours, 8), tf.greater_equal(batch_hours, 18))
    nighttime_mask = tf.cast(nighttime_mask, tf.float32)

    mse_loss = tf.keras.losses.MSE(y_true, y_pred)
    penalty_factor = 50.0
    weighted_loss = mse_loss * (1 + (penalty_factor - 1) * nighttime_mask)

    return tf.reduce_mean(weighted_loss)




In [229]:
classifier = Sequential([
    Dense(128, activation="relu", input_shape=(X_train.shape[1],)),
    Dense(64, activation="relu"),
    Dense(32, activation="relu"),
    Dense(1, activation="sigmoid")  # Binary classification
])
classifier.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
classifier.fit(X_train, y_train_class, epochs=10, batch_size=16, validation_data=(X_test, y_test_class))


Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


438/438 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.7583 - loss: 0.4923 - val_accuracy: 0.9229 - val_loss: 0.1819
Epoch 2/10
438/438 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.9228 - loss: 0.1666 - val_accuracy: 0.9429 - val_loss: 0.1512
Epoch 3/10
438/438 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9447 - loss: 0.1319 - val_accuracy: 0.9498 - val_loss: 0.1271
Epoch 4/10
438/438 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9418 - loss: 0.1280 - val_accuracy: 0.9366 - val_loss: 0.1397
Epoch 5/10
438/438 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9548 - loss: 0.1058 - val_accuracy: 0.9281 - val_loss: 0.1803
Epoch 6/10
438/438 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9519 - loss: 0.1117 - val_accuracy: 0.9526 - val_loss: 0.1064
Epoch 7/10
438/438 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9544 - loss: 0.1062 - val_accuracy: 0.9441 - val_loss: 0.1168
Epoch 8/10
438/438 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.9537 - loss: 0.1020 - val_accuracy: 0.9441 - val_

In [230]:
classifier.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
classifier.fit(X_train, y_train_class, epochs=10, batch_size=8, validation_data=(X_test, y_test_class))

Epoch 1/10
876/876 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.9543 - loss: 0.1033 - val_accuracy: 0.9509 - val_loss: 0.1208
Epoch 2/10
876/876 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.9528 - loss: 0.1054 - val_accuracy: 0.9612 - val_loss: 0.1069
Epoch 3/10
876/876 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9600 - loss: 0.0962 - val_accuracy: 0.9595 - val_loss: 0.1040
Epoch 4/10
876/876 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9529 - loss: 0.0991 - val_accuracy: 0.9406 - val_loss: 0.1238
Epoch 5/10
876/876 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9579 - loss: 0.1003 - val_accuracy: 0.9566 - val_loss: 0.0952
Epoch 6/10
876/876 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9600 - loss: 0.0926 - val_accuracy: 0.9578 - val_loss: 0.1009
Epoch 7/10
876/876 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9633 - loss: 0.0908 - val_accuracy: 0.9555 - val_loss: 0.1093
Epoch 8/10
876/876 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.9620 - loss: 0.0872 - val_accuracy: 0.

In [231]:
# Regressor Model (Predicts DNI_A for nonzero cases)
regressor = Sequential([
    Dense(128, activation="relu", input_shape=(X_train_reg.shape[1],)),
    Dense(64, activation="relu"),
    Dense(1, activation="linear")
])

regressor.compile(optimizer="adam", loss="mse", metrics=["mae"])
regressor.fit(X_train_reg, y_train_reg, epochs=10, batch_size=8, validation_data=(X_test_reg, y_test_reg))


Epoch 1/10
393/393 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 133841.2656 - mae: 300.8631 - val_loss: 45228.9531 - val_mae: 174.5505
Epoch 2/10
393/393 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 46662.7188 - mae: 180.1399 - val_loss: 42928.8906 - val_mae: 170.6913
Epoch 3/10
393/393 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 44727.5156 - mae: 176.2541 - val_loss: 42074.1250 - val_mae: 168.2758
Epoch 4/10
393/393 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 45400.7734 - mae: 175.8418 - val_loss: 41937.3516 - val_mae: 167.1936
Epoch 5/10
393/393 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 44160.3242 - mae: 174.2978 - val_loss: 41596.2461 - val_mae: 167.3587
Epoch 6/10
393/393 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 42847.3164 - mae: 170.7074 - val_loss: 41304.8594 - val_mae: 166.2441
Epoch 7/10
393/393 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 43038.9141 - mae: 171.1207 - val_loss: 41226.8164 - val_mae: 166.0867
Epoch 8/10
393/393 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 42297.6250 - mae: 168.9143 - 

In [232]:
# Predict probability of nonzero irradiance
is_daytime = classifier.predict(X_test)

# Predict irradiance only for nonzero cases
irradiance_predictions = regressor.predict(X_test)

# Final predictions: Apply classifier decision
y_pred = np.where(is_daytime > 0.48, irradiance_predictions, 0)  # 0 for nighttime

# Convert predictions into a DataFrame with timestamps
predictions_df = timestamps_test.copy()
predictions_df["DNI_A"] = y_pred  # Assign predicted irradiance

# Print first 100 predictions with date and hour
print(predictions_df.head(100))

# Save predictions to CSV
predictions_df.to_csv("solar_irradiance_predictions.csv", index=False)



55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
      Month  Day  Hour       DNI_A
6056      9   10     8  314.798737
5556      8   20    12  321.666626
5990      9    7    14  312.274628
7674     11   16    18    0.000000
3319      5   19     7  363.061371
...     ...  ...   ...         ...
6006      9    8     6    0.000000
8063     12    2    23    0.000000
14        1    1    14  463.148132
5306      8   10     2    0.000000
8524     12   22     4    0.000000

[100 rows x 4 columns]


In [233]:
# Save the classifier model
classifier.save("solar_classifier.keras")

# Save the regressor model
regressor.save("solar_regressor.keras")

In [234]:
#  Load the scalers used during training

with open("scaler_X.pkl", "rb") as f:
    scaler_X = pickle.load(f)
with open("scaler_y.pkl", "rb") as f:
    scaler_y = pickle.load(f)

In [235]:
# Generate timestamps for the first week of March 2025 (hourly)
date_range = pd.date_range(start="2025-03-01", end="2025-03-07 23:00:00", freq="H")
future_df = pd.DataFrame({"datetime": date_range})

<ipython-input-235-0e223ba96d09>:2: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  date_range = pd.date_range(start="2025-03-01", end="2025-03-07 23:00:00", freq="H")


In [236]:
#  Feature Engineering
future_df["day"] = future_df["datetime"].dt.day
future_df["month"] = future_df["datetime"].dt.month
future_df["hour"] = future_df["datetime"].dt.hour
future_df.drop(columns=["datetime"], inplace=True)

In [237]:
# Generate random values
future_df["Temperature_A"] = np.random.uniform(27, 30, size=168)  # Temperature between 15°C to 30°C
future_df["Pressure_A"] = np.random.uniform(1000, 1050, size=168)  # Pressure between 950-1050 hPa
future_df["Surface Albedo_A"] = np.random.uniform(0.1, 0.5, size=168)  # Albedo between 0.1 - 0.5
future_df["Cloud Type_A"] = np.random.randint(0, 4, size=168)  # Cloud types (0-3)

# Merge the first week’s random values with `future_df`
#future_df = future_df.merge(future_df, on=["Month", "Day"], how="left")

# Time-based encoding (sin/cos transformation)
#future_df["hour_sin"] = np.sin(2 * np.pi * future_df["hour"] / 24)
#future_df["hour_cos"] = np.cos(2 * np.pi * future_df["hour"] / 24)

# Select the same features used in training
#X_future = future_df[['hour_sin', 'hour_cos', 'Temperature_A', 'Pressure_A', 'day', 'Month', 'Surface Albedo_A', 'Cloud Type_A']].values

In [238]:
#  Select the same features used in training
X_future = future_df[['hour', 'Temperature_A','Pressure_A', 'day','month', 'Surface Albedo_A', 'Cloud Type_A']].values
#  Scale future data using the trained scaler
X_future_scaled = scaler_X.transform(X_future)
print(X_future_scaled)


[[ 0.          0.66374893  0.88545454 ...  0.18181818  2.04845587
   0.22222222]
 [ 0.04347826  0.63037787  0.36406826 ...  0.18181818  5.12461426
   0.11111111]
 [ 0.08695652  0.61842546  1.57719148 ...  0.18181818  6.72161999
   0.33333333]
 ...
 [ 0.91304348  0.65767429  1.54495812 ...  0.18181818  5.88668455
   0.        ]
 [ 0.95652174  0.59869559  0.38635018 ...  0.18181818  4.81414367
   0.11111111]
 [ 1.          0.646514    1.60582428 ...  0.18181818 -0.74496513
   0.11111111]]


In [239]:
# Predict probability of nonzero irradiance
is_daytime_future = classifier.predict(X_future_scaled)

# Predict irradiance only for nonzero cases
irradiance_future_scaled = regressor.predict(X_future_scaled)

# Apply classification decision: Set nighttime values to 0
y_future_scaled = np.where(is_daytime_future > 0.48, irradiance_future_scaled, 0)

# Inverse scale predictions
y_future = scaler_y.inverse_transform(y_future_scaled.reshape(-1, 1))

# Set small irradiance values to 0
y_future[y_future < 15] = 0  # Adjust threshold if needed


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 


In [240]:
# Save predictions to DataFrame
future_df[['DNI_A']] = y_future

#  Save to CSV
future_df.to_csv("Future_Solar_Predictions_March2025.csv", index=False)

#Display first 10 predictions
print(future_df.head(10))

   day  month  hour  Temperature_A   Pressure_A  Surface Albedo_A  \
0    1      3     0      29.838215  1017.563636          0.242423   
1    1      3     1      28.750318  1001.922048          0.396231   
2    1      3     2      28.360670  1038.315744          0.476081   
3    1      3     3      28.434308  1010.820970          0.417147   
4    1      3     4      27.302557  1013.921707          0.232054   
5    1      3     5      28.894407  1014.005403          0.135589   
6    1      3     6      27.870433  1012.056765          0.386715   
7    1      3     7      28.237076  1025.996088          0.160713   
8    1      3     8      28.799235  1011.601873          0.287388   
9    1      3     9      27.598366  1029.550996          0.201134   

   Cloud Type_A       DNI_A  
0             2    0.000000  
1             1    0.000000  
2             3    0.000000  
3             0    0.000000  
4             2    0.000000  
5             3    0.000000  
6             0  472.634888  
